<a href="https://colab.research.google.com/github/sarath11999/colab3d/blob/main/SolarModelSarathclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, Subset
from torch import nn
import math

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
large_input_paths = ['/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/measurements/measured_active_power.csv',
                     '/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/measurements/measured_reactive_power.csv',
                     '/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/measurements/measured_voltage_angles.csv',
                     '/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/measurements/measured_voltage_magnitudes.csv']

large_required_input_paths = ['/content/drive/MyDrive/newfolder/measured_active_power_filtered.csv',
                              '/content/drive/MyDrive/newfolder/measured_reactive_power_filtered.csv',
                              '/content/drive/MyDrive/newfolder/measured_voltage_angles_filtered.csv',
                              '/content/drive/MyDrive/newfolder/measured_voltage_magnitudes_filtered.csv']

large_target_paths = ['/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/voltages/actual_voltage_magnitudes.csv',
                      '/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/voltages/actual_voltage_angles.csv']

In [11]:
large_day14_input_paths = ['/content/drive/MyDrive/day14/measured_active_power.csv',
                           '/content/drive/MyDrive/day14/measured_reactive_power.csv',
                           '/content/drive/MyDrive/day14/measured_voltage_angles.csv',
                           '/content/drive/MyDrive/day14/measured_voltage_magnitudes.csv']

In [12]:
dates={'Topology 1': {
        'start': pd.to_datetime('4/1/2018 7:00'),
        'end': pd.to_datetime('6/14/2018 18:45')
    },
    'Topology 2': {
        'start': pd.to_datetime('6/15/2018 7:00'),
        'end': pd.to_datetime('6/29/2018 18:45')
    }}

In [13]:
topology_df=pd.read_csv('/content/drive/MyDrive/DOE_3D_Solar_Visibility/Historical_Data/large-system/measurements/measured_active_power.csv', index_col='Timestep')
topology_df['Topology']=0
topology_df=topology_df[['Topology']]

In [15]:
topology_df.index = pd.to_datetime(topology_df.index)
topology_df.loc[dates['Topology 1']['start']:dates['Topology 1']['end'],'Topology']=1
topology_df.loc[dates['Topology 2']['start']:dates['Topology 2']['end'],'Topology']=2
topology_df.to_csv('topology.csv')
topology_df

,Topology
Timestep,
2018-04-01 07:00:00,1
2018-04-01 07:15:00,1
2018-04-01 07:30:00,1
2018-04-01 07:45:00,1
2018-04-01 08:00:00,1
...,...
2018-06-29 17:45:00,2
2018-06-29 18:00:00,2
2018-06-29 18:15:00,2


In [16]:
class CustomDatasetTrainTest(Dataset):
    def __init__(self, inputs, targets, input_scalers=None):
        self.inputs_df = [self.encode_time_and_dropzeros(input_path) for input_path in inputs]
        self.targets_df = [self.encode_time_and_dropzeros(target_path, target=True) for target_path in targets]

        # Initialize scalers if they are not passed
        if input_scalers is None:
            self.input_scalers = [StandardScaler() for _ in inputs]
        else:
            self.input_scalers = input_scalers

        # Fit scalers only on inputs and targets
        self._fit_scalers()

    def encode_time_and_dropzeros(self, path, target=False):
        df = pd.read_csv(path, parse_dates=True, index_col='Timestep')

        if not target:
            # Ensure the index is a DatetimeIndex
            if not isinstance(df.index, pd.DatetimeIndex):
                raise ValueError("The DataFrame index must be a DatetimeIndex.")

            # Extract relevant time information
            hours = df.index.hour
            day_of_week = df.index.dayofweek  # Monday=0, Sunday=6
            day_of_year = df.index.dayofyear   # 1 through 365 (or 366 for leap year)
            day_of_month = df.index.day        # 1 through 31

            # Calculate sine and cosine encodings
            df['sin_hour'] = np.sin(2 * np.pi * hours / 24)
            df['cos_hour'] = np.cos(2 * np.pi * hours / 24)

            df['sin_day_of_week'] = np.sin(2 * np.pi * day_of_week / 7)
            df['cos_day_of_week'] = np.cos(2 * np.pi * day_of_week / 7)

            df['sin_day_of_year'] = np.sin(2 * np.pi * day_of_year / 365)
            df['cos_day_of_year'] = np.cos(2 * np.pi * day_of_year / 365)

            df['sin_day_of_month'] = np.sin(2 * np.pi * day_of_month / 31)
            df['cos_day_of_month'] = np.cos(2 * np.pi * day_of_month / 31)

            # Drop columns where all values are zero
            df = df.loc[:, (df != 0).any(axis=0)]

        else:
            # Convert target values
            if df.shape[1] == 1:  # Ensure there's only one column
                df.iloc[:, 0] = df.iloc[:, 0].replace({1: 0, 2: 1})
            else:
                raise ValueError("Target DataFrame must have exactly one column.")

        return df

    def _fit_scalers(self):
        """Fit the scalers on the inputs and targets"""
        for i, input_df in enumerate(self.inputs_df):
            self.input_scalers[i].fit(input_df)


    def standardize_data(self, df, scaler):
        """Standardize the given DataFrame using the scaler"""
        return pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

    def __len__(self):
        return len(self.inputs_df[0])

    def __getitem__(self, index):
        # Standardize inputs
        inputs = [torch.tensor(self.standardize_data(input_df.iloc[[index]], self.input_scalers[i]).values[0], dtype=torch.float32)
                  for i, input_df in enumerate(self.inputs_df)]

        # Standardize targets
        targets = [torch.tensor((target_df.iloc[[index]]).values[0], dtype=torch.float32)
                   for j, target_df in enumerate(self.targets_df)]

        return (*inputs, *targets)


def train_test_split(dataset, test_days=10):
    test_length = test_days * 48
    train_length = len(dataset) - test_length
    train_data = Subset(dataset, range(train_length))
    test_data = Subset(dataset, range(train_length, len(dataset)))
    return train_data, test_data

In [17]:
classification_target=['/content/topology.csv']

 **Small Dataset**

In [18]:
full_dataset= CustomDatasetTrainTest(inputs=large_input_paths, targets=classification_target)

In [19]:
# Print means and standard deviations for input data scalers
print("Input Data Scalers:")
for i, scaler in enumerate(full_dataset.input_scalers):
    print(f"Input {i + 1} Mean: {scaler.mean_}")
    print(f"Input {i + 1} Standard Deviation: {scaler.scale_}")

Input Data Scalers:
Input 1 Mean: [-1.44335092e+03 -1.49420661e+03 -1.34262780e+03 ... -6.23197782e-01
  2.23665022e-03 -3.31058882e-02]
Input 1 Standard Deviation: [3.92447686e+02 3.22223205e+02 3.63282991e+02 ... 3.10411394e-01
 7.18478552e-01 6.94757201e-01]
Input 2 Mean: [-5.13510498e+02 -4.17609852e+02 -4.57630593e+02 ... -6.23197782e-01
  2.23665022e-03 -3.31058882e-02]
Input 2 Standard Deviation: [138.77467352 116.9518031  130.8085159  ...   0.31041139   0.71847855
   0.6947572 ]
Input 3 Mean: [-1.68777611e-07 -2.09439530e+00  2.09439494e+00 -2.11518290e+00
  2.08158592e+00 -1.05399365e+00  3.11348603e+00 -1.06222654e+00
 -1.05809812e+00 -2.44730140e-02 -1.06330221e+00 -8.33333333e-02
 -6.32979509e-01  1.08325324e-02  2.47245482e-03  6.53420383e-01
 -6.23197782e-01  2.23665022e-03 -3.31058882e-02]
Input 3 Standard Deviation: [5.70756596e-08 4.40611548e-08 5.18252036e-08 3.53705795e-03
 6.76500725e-03 2.02119647e-03 1.02605231e-02 6.37380770e-03
 6.15329850e-03 9.01564804e-03 7.3

In [20]:
# Split dataset
train_data, test_data = train_test_split(full_dataset, test_days=10)

In [21]:
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

In [22]:
for i, data in enumerate(train_loader):
  input1, input2, input3, input4, target = data
  print(target[:10])
  break

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


In [23]:
class Solar3DClassificationModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, output_dim, custom_linear_layers_dim=None, max_len = 256):
        super(Solar3DClassificationModel, self).__init__()
        self.input_proj1 = nn.Linear(input_dim[0], d_model)
        self.input_proj2 = nn.Linear(input_dim[1], d_model)
        self.input_proj3 = nn.Linear(input_dim[2], d_model)
        self.input_proj4 = nn.Linear(input_dim[3], d_model)


        encoder_layer1 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder1 = nn.TransformerEncoder(encoder_layer1, num_layers=num_encoder_layers)

        encoder_layer2 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder2 = nn.TransformerEncoder(encoder_layer2, num_layers=num_encoder_layers)

        encoder_layer3 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder3 = nn.TransformerEncoder(encoder_layer3, num_layers=num_encoder_layers)

        encoder_layer4 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder4 = nn.TransformerEncoder(encoder_layer4, num_layers=num_encoder_layers)

        if custom_linear_layers_dim:
            custom_layers = []
            for dim in custom_linear_layers_dim:
                custom_layers.append(nn.Linear(d_model, dim))
                custom_layers.append(nn.ReLU())
                d_model = dim
            self.custom_layers1 = nn.Sequential(*custom_layers)
            self.custom_layers2 = nn.Sequential(*custom_layers)
        else:
            self.custom_layers1 = None
            self.custom_layers2 = None

        self.final_proj = nn.Linear(d_model, output_dim)

        self.init_weights()

    def init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src1, src2, src3, src4):
        src1 = self.input_proj1(src1)
        src2 = self.input_proj2(src2)
        src3 = self.input_proj3(src3)
        src4 = self.input_proj4(src4)

        transformer_output1 = self.transformer_encoder1(src1)
        transformer_output2 = self.transformer_encoder2(src2)
        transformer_output3 = self.transformer_encoder3(src3)
        transformer_output4 = self.transformer_encoder4(src4)

        transformer_output = (transformer_output1 + transformer_output2 +
                              transformer_output3 + transformer_output4) / 4

        if self.custom_layers1:
            transformer_output = self.custom_layers1(transformer_output)
        else:
            transformer_output = transformer_output

        transformer_output = self.final_proj(transformer_output)

        return transformer_output

In [63]:
for training_data in train_loader:
  print(f"The shapes are: {[data.shape for data in training_data]}")
  break

The shapes are: [torch.Size([256, 3260]), torch.Size([256, 3266]), torch.Size([256, 19]), torch.Size([256, 1711]), torch.Size([256, 1])]


In [5]:
model01= Solar3DClassificationModel(
    input_dim=[3260, 3266, 19, 1711],  # dimensions of each input
    d_model=256,
    nhead=32,
    num_encoder_layers=1,
    dim_feedforward=4,
    dropout=0.7,
    output_dim= 1,  # dimensions of each output
    custom_linear_layers_dim=[128, 512, 128, 512, 128]
)

In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model01.parameters(), lr=0.01)

In [86]:
model01.eval()
with torch.inference_mode():
  for data in train_loader:
    input1, input2, input3, input4, target = data
    output = model01(input1, input2, input3, input4)
    probabilities = torch.sigmoid(output).squeeze()
    print(probabilities)
    loss = criterion(output, target.float())
    print(loss)
    break

tensor([0.5199, 0.5199, 0.5016, 0.5131, 0.5276, 0.5046, 0.4989, 0.5102, 0.5242,
        0.5391, 0.5072, 0.5029, 0.4931, 0.5035, 0.5212, 0.5035, 0.5054, 0.5182,
        0.5364, 0.5091, 0.5135, 0.4886, 0.5144, 0.5136, 0.5070, 0.5070, 0.5250,
        0.5093, 0.5131, 0.5277, 0.5031, 0.5067, 0.5092, 0.5216, 0.5302, 0.5138,
        0.4955, 0.5177, 0.5127, 0.5312, 0.5050, 0.5204, 0.5120, 0.4975, 0.5118,
        0.4992, 0.5358, 0.5077, 0.5302, 0.4977, 0.5058, 0.5184, 0.5329, 0.5258,
        0.5030, 0.4818, 0.5121, 0.5023, 0.5129, 0.5041, 0.4940, 0.5061, 0.5059,
        0.5141, 0.5124, 0.5322, 0.5177, 0.4893, 0.5155, 0.4944, 0.5157, 0.5062,
        0.4935, 0.5066, 0.5208, 0.4952, 0.5128, 0.5581, 0.4987, 0.5135, 0.5231,
        0.5081, 0.5159, 0.5212, 0.4994, 0.4959, 0.5009, 0.5123, 0.5141, 0.4995,
        0.5230, 0.5199, 0.5182, 0.5087, 0.4980, 0.5327, 0.4984, 0.5161, 0.5144,
        0.5316, 0.5118, 0.5189, 0.5344, 0.5219, 0.5036, 0.5045, 0.5268, 0.4976,
        0.5228, 0.4978, 0.5160, 0.4960, 

In [97]:
# # Ensure to use the appropriate device (CPU or GPU)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize lists to record losses, outputs, and accuracies
train_losses = []
test_losses = []
train_op = []
test_op = []
train_accuracies = []
test_accuracies = []

# Load pre-trained weights if available
# model01.load_state_dict(torch.load('model_weights.pth'))

epochs = 1
for epoch in range(epochs):
    model01.train()
    total_train_loss = 0  # Track total loss for this epoch
    correct_train = 0
    total_train = 0

    for data in train_loader:
        input1, input2, input3, input4, target = data
        output = model01(input1, input2, input3, input4)
        probabilities = torch.sigmoid(output).squeeze()
        train_predictions = (probabilities > 0.5).long()
        optimizer.zero_grad()  # Zero the gradients
        if epoch == epochs - 1:
            train_op.extend(train_predictions.detach().cpu().numpy())

        # Calculate loss
        loss1 = criterion(output, target.float())
        loss1.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_train_loss += loss1.item()  # Accumulate loss

        # Calculate accuracy
        correct_train += (train_predictions == target).sum().item()
        total_train += target.size(0)

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Perform inference
    model01.eval()
    total_test_loss = 0
    correct_test = 0
    total_test = 0

    with torch.no_grad():  # Disable gradient calculation
        for data in test_loader:
            input1, input2, input3, input4, target = data
            output = model01(input1, input2, input3, input4)
            probabilities = torch.sigmoid(output).squeeze()
            test_predictions = (probabilities > 0.5).long()
            if epoch == epochs - 1:
                test_op.extend(test_predictions.detach().cpu().numpy())

            # Calculate loss
            loss2 = criterion(output, target.float())
            total_test_loss += loss2.item()  # Accumulate loss

            # Calculate accuracy
            correct_test += (test_predictions == target).sum().item()
            total_test += target.size(0)

    avg_test_loss = total_test_loss / len(test_loader)
    test_accuracy = correct_test / total_test
    test_losses.append(avg_test_loss)
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"Epoch {epoch + 1}/{epochs}, Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy * 100:.2f}%")

tensor([0.1095, 0.1295, 0.1279, 0.1097, 0.1081, 0.0950, 0.1223, 0.1031, 0.0991,
        0.0946, 0.1174, 0.1361, 0.1173, 0.1125, 0.1131, 0.0936, 0.1153, 0.0940,
        0.1487, 0.1059, 0.1185, 0.1083, 0.1455, 0.1342, 0.1613, 0.1314, 0.1432,
        0.1726, 0.1487, 0.1928, 0.1778, 0.1817, 0.1810, 0.1816, 0.1909, 0.1909,
        0.1892, 0.1915, 0.1816, 0.1968, 0.1923, 0.1858, 0.1881, 0.1804, 0.1720,
        0.1611, 0.1755, 0.1727, 0.1370, 0.1220, 0.1538, 0.1605, 0.1607, 0.1504,
        0.1499, 0.1364, 0.1572, 0.1629, 0.1710, 0.1671, 0.1888, 0.1837, 0.1468,
        0.1649, 0.1533, 0.1261, 0.1537, 0.1728, 0.1767, 0.1815, 0.1871, 0.1710,
        0.1859, 0.1923, 0.1942, 0.1963, 0.1979, 0.2032, 0.1992, 0.2041, 0.1981,
        0.1921, 0.1884, 0.1827, 0.1794, 0.1799, 0.1626, 0.1572, 0.1525, 0.1496,
        0.1564, 0.1585, 0.1568, 0.1496, 0.1599, 0.1566, 0.1509, 0.1488, 0.1522,
        0.1570, 0.1607, 0.1718, 0.1942, 0.1990, 0.2001, 0.1981, 0.1990, 0.1911,
        0.1874, 0.1886, 0.1933, 0.1920, 

In [39]:
def actual_output_data(output):
    outputs_final=[]
    for i in output:
      if i == 0:
        outputs_final.append(1)
      else:
        outputs_final.append(2)
    return pd.DataFrame({"Topology":outputs_final})

In [102]:
actual_output_data(test_op)

,Topology
0,1
1,1
2,1
3,1
4,1
...,...
475,1
476,1
477,1
478,1


In [24]:
# Define paths for future data
large_day14_input_paths = [
    '/content/drive/MyDrive/day14/measured_active_power.csv',
    '/content/drive/MyDrive/day14/measured_reactive_power.csv',
    '/content/drive/MyDrive/day14/measured_voltage_angles.csv',
    '/content/drive/MyDrive/day14/measured_voltage_magnitudes.csv'
]

In [25]:
class CustomDatasetForRealData(Dataset):
    def __init__(self, inputs, input_scalers=None):
        self.inputs_df = [self.encode_time_and_dropzeros(input_path) for input_path in inputs]

        # Initialize scalers if they are not passed
        if input_scalers is None:
            self.input_scalers = [StandardScaler() for _ in inputs]
        else:
            self.input_scalers = input_scalers

        # Fit scalers only on inputs
        self._fit_scalers()

    def encode_time_and_dropzeros(self, path):
        df = pd.read_csv(path, parse_dates=True, index_col='Timestep')

        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("The DataFrame index must be a DatetimeIndex.")

        # Extract relevant time information
        hours = df.index.hour
        day_of_week = df.index.dayofweek  # Monday=0, Sunday=6
        day_of_year = df.index.dayofyear   # 1 through 365 (or 366 for leap year)
        day_of_month = df.index.day        # 1 through 31

        # Calculate sine and cosine encodings
        df['sin_hour'] = np.sin(2 * np.pi * hours / 24)
        df['cos_hour'] = np.cos(2 * np.pi * hours / 24)

        df['sin_day_of_week'] = np.sin(2 * np.pi * day_of_week / 7)
        df['cos_day_of_week'] = np.cos(2 * np.pi * day_of_week / 7)

        df['sin_day_of_year'] = np.sin(2 * np.pi * day_of_year / 365)
        df['cos_day_of_year'] = np.cos(2 * np.pi * day_of_year / 365)

        df['sin_day_of_month'] = np.sin(2 * np.pi * day_of_month / 31)
        df['cos_day_of_month'] = np.cos(2 * np.pi * day_of_month / 31)

        df = df.loc[:, (df != 0).any(axis=0)]
        return df

    def _fit_scalers(self):
        """Fit the scalers on the inputs"""
        for i, input_df in enumerate(self.inputs_df):
            self.input_scalers[i].fit(input_df)

    def standardize_data(self, df, scaler):
        """Standardize the given DataFrame using the scaler"""
        return pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

    def __len__(self):
        return len(self.inputs_df[0])

    def __getitem__(self, index):
        # Standardize inputs
        inputs = [torch.tensor(self.standardize_data(input_df.iloc[[index]], self.input_scalers[i]).values[0], dtype=torch.float32)
                  for i, input_df in enumerate(self.inputs_df)]

        # Return as a tuple
        return tuple(inputs)


In [26]:
real_datasets=CustomDatasetForRealData(inputs=large_day14_input_paths)

In [27]:
final_test_loader = DataLoader(real_datasets, batch_size=256, shuffle=False)

In [28]:
for training_data in final_test_loader :
  print(f"The shapes are: {[data.shape for data in training_data]}")
  break

The shapes are: [torch.Size([48, 2892]), torch.Size([48, 2895]), torch.Size([48, 19]), torch.Size([48, 433])]


In [29]:
model02= Solar3DClassificationModel(
    input_dim=[2892, 2895,  19, 433],  # dimensions of each input
    d_model=256,
    nhead=32,
    num_encoder_layers=1,
    dim_feedforward=4,
    dropout=0.7,
    output_dim= 1,  # dimensions of each output
    custom_linear_layers_dim=[128, 512, 128, 512, 128]
)

In [34]:
# Ensure to use the appropriate device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [35]:
device=torch.device('cpu')
device

device(type='cpu')

In [36]:
# Perform inference
model02.eval()
output = []
with torch.inference_mode():  # Disable gradient calculation
    for data in final_test_loader:
        input1, input2, input3, input4 = [x.to(device) for x in data]

        # Perform forward pass
        batch_output = model02(input1, input2, input3, input4)
        probabilities = torch.sigmoid(batch_output).squeeze()
        test_predictions = (probabilities > 0.5).long()
        # Append outputs to lists
        output.append(test_predictions.detach().cpu().numpy())

# Concatenate all batches into a single array
output = np.concatenate(output, axis=0)

In [37]:
output

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [40]:
day14_output=actual_output_data(output)
day14_output

,Topology
0,2
1,2
2,2
3,2
4,2
5,2
6,2
7,2
8,2
9,2


In [48]:
index_day14=pd.read_csv('/content/drive/MyDrive/day14/measured_active_power.csv',index_col='Timestep').index
day14_output.index=index_day14
day14_output.to_csv('topology_changes.csv')
day14_output

,Topology
Timestep,
2018-10-28 07:00:00,2
2018-10-28 07:15:00,2
2018-10-28 07:30:00,2
2018-10-28 07:45:00,2
2018-10-28 08:00:00,2
2018-10-28 08:15:00,2
2018-10-28 08:30:00,2
2018-10-28 08:45:00,2
2018-10-28 09:00:00,2
